In [1]:
%pip install jaro-winkler textdistance pyphonetics wordfreq cyhunspell

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 56.8 MB 29 kB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 235 kB 54.9 MB/s 
     |████████████████████████████████| 273 kB 89.1 MB/s 
     |████████████████████████████████| 173 kB 86.9 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.5.1-py3-none-any.whl size=56830992 sha256=e9914322b1fece78bc85cc4214f82a98a251efd38b7fc7c042c14d7fc66cfdb3
  Stored in directory: /tmp/xdg_cache/pip/wheels/d8/d4/2d/741daa49d00231b4c5afbec15e79f361d9e76961f1a5fc02e5
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41913 sha256=a35a51dfdcbf6d25ab51927f29927ef6ca4291db1f0714d5684d104bb9d2724c
  Stored in directory: /tmp/xdg_cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
  Created wheel for langcodes: fi

In [2]:
import nltk
nltk.download('punkt', quiet=True)

True

In [3]:
import numpy as np
from nltk.tokenize import word_tokenize
from hunspell import Hunspell
from pyphonetics import RefinedSoundex

from jaro import jaro_winkler_metric
from textdistance import levenshtein
from wordfreq import word_frequency

In [ ]:
def sort_candidates(word, candidates):
    if len(candidates) == 0:
            return []
    
    

In [8]:
hunspell = Hunspell('en_US')
soundex = RefinedSoundex()


def rank_suggestions(word, suggestions, n_suggestions):
    features = np.zeros((len(suggestions), 4))
    for i, suggestion in enumerate(suggestions):
        text_edit_distance = levenshtein(word, suggestion)
        phoneme_edit_distance = soundex.distance(word, suggestion, metric='levenshtein')
        suggestion_prob = word_frequency(suggestion, 'en')
        jw_distance = jaro_winkler_metric(word, suggestion)
        
        features[i] = np.array([text_edit_distance, phoneme_edit_distance, 1 - suggestion_prob, jw_distance])
    
    norm_features = ((features - features.min(axis=0)) / (features.max(axis=0) - features.min(axis=0) + 1e-9))
    weights = norm_features.sum(axis=1)
    ranked_suggestions = [suggestions[i] for i in np.argsort(weights)]
    return ranked_suggestions[:n_suggestions]


def spellcheck(text: str, n_suggestions: int):
    words = word_tokenize(text, 'english')
    
    for i, word in enumerate(words):
        if not word.isalpha():
            continue
        if hunspell.spell(word):
            continue
        
        suggestions = hunspell.suggest(word)
        print(f'Word "{word}" at position {i} is probably misspelled.')
        if not suggestions:
            print(f'No fixes available.')
        else:
            print('Available fixes:')
            for suggestion in rank_suggestions(word, suggestions, n_suggestions):
                print(f'- {suggestion}')
        print()

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: hunspell
  warnings.warn(message)


In [12]:
text = 'The Hamming distence is named after Richard Hamming, who intrdced the concept in his fundamental paper on Hamming codes, Error detecting and error corectin codes, in 1950. Hamming weight analysis of bits is used in several disciplines including infzzmation theory, coding theory, and cryptography.'
n_suggestions = 4
spellcheck(text, n_suggestions)

Word "distence" at position 2 is probably misspelled.
Available fixes:
- distance
- existence
- insistence
- distend

Word "intrdced" at position 10 is probably misspelled.
Available fixes:
- introduced
- intercede
- interceded

Word "corectin" at position 25 is probably misspelled.
Available fixes:
- correction
- corrector
- incorrect
- corrective

Word "infzzmation" at position 42 is probably misspelled.
Available fixes:
- information
- inflammation
- intimation



/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: hunspell
  warnings.warn(message)
